Some of the features that has and hasn't been done:

- [X] Display points from all views simultaneously.
- [ ] Give color according to vehicle type
- [ ] Display the CCTV views
- [ ] Display the bounding boxes to CCTV views.

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import time

region_data = pd.read_csv('regions.csv')
region_data

,cctv_folder_parent,map_image_name
0,SIMPANG 3 DPR,SIMPANG 3 DPR.jpeg
1,SIMPANG 4 CUNGKING,SIMPANG 4 CUNGKING.jpeg
2,SIMPANG 4 PATUNG KUDA,SIMPANG 4 PATUNG KUDA.jpeg
3,SIMPANG 5 PUSAT KOTA,SIMPANG 5 BANYUWANGI.jpeg


In [3]:
cctv_data = pd.read_csv('cctv_views.csv')
cctv_data

,region_name,cctv_view_name
0,SIMPANG 3 DPR,dpr selatan
1,SIMPANG 3 DPR,dpr timur
2,SIMPANG 3 DPR,dpr utara
3,SIMPANG 4 PATUNG KUDA,patungkuda barat
4,SIMPANG 4 PATUNG KUDA,patungkuda selatan
5,SIMPANG 4 PATUNG KUDA,patungkuda utara


In [28]:
# Define a CCTV path of image whose region has been detected
# Define the map path as well.

# You can freely change this depending on the condition of your database.
region_name = "SIMPANG 4 PATUNG KUDA"
cctv_range = {'start':0, 'end': 252}
cctv_time = '1200'

region_cctv_views = cctv_data[cctv_data['region_name'] == region_name]['cctv_view_name'].values.tolist()

region_map_path = region_data[region_data['cctv_folder_parent'] == region_name]
region_map_path = 'map/' + region_map_path['map_image_name'].values.tolist()[0]

region_coords = pd.read_csv('coords/cctv/{:}.csv'.format(region_name))

print("CCTV VIEWS :", region_cctv_views)
print("MAP PATH   :", region_map_path)
print("REGION COORDINATES:")
region_coords

CCTV VIEWS : ['patungkuda barat', 'patungkuda selatan', 'patungkuda utara']
MAP PATH   : map/SIMPANG 4 PATUNG KUDA.jpeg
REGION COORDINATES:


,view,pt1a,pt1b,pt1c,pt1d,pt2a,pt2b,pt2c,pt2d,pt3a,pt3b,pt3c,pt3d
0,patungkuda selatan,97 273,363 236,1074 476,187 770,0 0,1080 0,1080 720,0 720,400 717,295 644,392 513,492 587
1,patungkuda barat,132 283,396 262,1080 505,211 720,0 0,1080 0,1080 720,0 720,364 296,475 236,578 379,458 450
2,patungkuda utara,510 170,668 168,664 437,0 390,0 0,1080 0,1080 720,0 720,648 167,748 239,598 437,497 362


In [5]:
map_image = cv2.imread(region_map_path)

In [6]:
def get_points(pandas_frame, view_name):
    polygons = {}
    row = pandas_frame[pandas_frame['view'] == view_name]
    point_id = ['a', 'b', 'c', 'd']
    polygon_id = ['1', '2', '3']
    for i in polygon_id:
        polygon = []
        for j in point_id:
            point_name = 'pt'+ i + j
            point_array = [int(p) for p in row[point_name].values.tolist()[0].split()]
            polygon.append(point_array)
        polygons[i] = np.float32(polygon)
    return polygons

In [7]:
def warpPerspectivePoints(pts, M):
    n = pts.shape[0]
    trans = np.hstack((pts, np.ones((n,1)))) # Add ones at the bottom of matrix
    trans = np.matmul(M, np.transpose(trans))
    return np.transpose(trans[0:2,:] / trans[2:])

##### Pseudocode:

For every CCTV views:

1. Eliminate non-vehicles point (can be obtained from the "object" header)
   For example, vehicles: truck, car, motorbike
1. For every bounding box vehicles:
   1. Find the bounding box center point
   1. Map the point according to transformation matrix M
1. Now, for displaying purposes:
   1. Draw the map and all the transformed points
   1. In new window, draw the CCTV image with boxes

In [29]:
coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

def eliminate_non_vehicle(series):
    detections = series['object_probs'].split(' ')
    for dtc in detections:
        if dtc in coco_vehicle_labels:
            return True
    return False

In [11]:
# Let's define a dictionary to store variables.
# These variables include precomputed value as well, such as
# the transformation matrix.

objs = {}
for i in range(len(region_cctv_views)):
    obj_dict = {}
    
    rctv = region_cctv_views[i]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))    

    # Get detections CSV file name length
    # It is assumed that all files has the same name length.
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])
    
    # Populate variables to the object dictionary.    
    obj_dict['rctv'] = rctv
    obj_dict['path_cctv_view'] = path_cctv_view
    obj_dict['pts1'] = pts1
    obj_dict['pts2'] = pts2
    obj_dict['filename_length'] = filename_length
    obj_dict['M'] = M
    objs[i] = obj_dict

In [33]:
ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button
finished = False

try:
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        # Copy the image. Later we will draw something.
        # We don't want the original to be overwritten.
        new_map_image = map_image.copy()
        
        for j in range(len(region_cctv_views)):                
            obj = objs[j]
            
            # Print current progress (for debug purposes)
            print('\r' + str(i) + '-' + obj['rctv'] + '   ', end='')
            
            M = obj['M']
            filename_length = obj['filename_length']
            path_cctv_view = obj['path_cctv_view']
            
            csv_file_name = str(i).zfill(filename_length)
            csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)

            # Read CSV
            objects = pd.read_csv(csv_path)
            
            # Eliminate non-vehicles object
            n_detection = objects.shape[0]
            if (n_detection > 0):
                objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]

            # For every bounding box vehicles, find the bounding box center point
            n_detection = objects.shape[0]
            if (n_detection > 0):
                centers = objects.apply(lambda r: [
                                  (r['left'] + r['right'])/2,
                                  (r['top'] + r['bottom'])/2
                                  ], axis=1)

                # Transform the points
                points_np = np.array([c for c in centers])
                points_transform = np.int32(warpPerspectivePoints(points_np, M))

                # Map the points to the image
                for pts_t in points_transform:
                    cv2.circle(new_map_image, tuple(pts_t), 9, (0,255,0), -1)
        
        cv2.imshow('Image', new_map_image)
        
        # Pauses and quit button purposes
        k = cv2.waitKey(25) & 0xFF
        finished = k == ord_q       
        if k == ord_p:
            while True:
                k = cv2.waitKey(25) & 0xFF
                finished = k == ord_q
                if k == ord_p or k == ord_q:
                    break
        if finished:
            break
finally:
    cv2.destroyAllWindows()

133-patungkuda utara     

---
##### From this point, the cells below are used for sandbox purposes only.

In [ ]:
# Try for single video frames first
try:
    rctv = region_cctv_views[2]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    print("VIEW:", rctv)
    print("PTS 1:", str(pts1).replace("\n", ""))
    print("PTS 2:", str(pts2).replace("\n", ""))

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))
    cv2.fillPoly(map_image_filled, [np.int32(pts_transform)], (255, 0, 0))
    
    print(pts_transform.tolist())

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    print("FIRST FILE:", file_ext_first_file)
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Try for single frame video first
    # Read CSV detection sequentially
    
    for i in range(45, cctv_range['end'] + 1):
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)

        # Read CSV
        objects = pd.read_csv(csv_path)
        print()
        print("OBJECTS:")
        print(objects)
        print()

        # Eliminate non-vehicles object
        def eliminate_non_vehicle(series):
            detections = series['object_probs'].split(' ')
            for dtc in detections:
                if dtc in coco_vehicle_labels:
                    return True
            return False
        
        objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]

        # Change string columns into numeric
        # -- Yeah, a little mistake I did when outputting CSV in darknet :( 
        #cols = ['left', 'right', 'top', 'bottom']    
        #for j in cols:
        #    objects[j] = pd.to_numeric(objects[j])

        # For every bounding box vehicles, find the bounding box center point    
        centers = objects.apply(lambda r: [
                          (r['left'] + r['right'])/2,
                          (r['top'] + r['bottom'])/2
                          ], axis=1)
        print("CENTERS:", str(centers).replace("\n", "").replace("  ", " "))

        # Transformation the points
        points_np = np.array([i for i in centers])
        points_transform = np.int32(warpPerspectivePoints(points_np, M))
        print("CENTER TRANSFORMED", str(points_transform).replace("\n", ""))

        # Map the points to the image
        for i in points_transform:
            cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)
        cv2.imshow('Image2', overlay_transform)

        cv2.imshow('Image', new_map_image)
        while True:
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        break
finally:
    cv2.destroyAllWindows()

In [34]:
# Now let's try it for all video frames.
try:
    rctv = region_cctv_views[1]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    map_image_filled = map_image.copy()
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Read CSV detection sequentially    
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        print('\r' + str(i), end='')
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
        
        # Read CSV
        objects = pd.read_csv(csv_path)
        
        # Do not progress if nothing is detected
        n_detection = objects.shape[0]
        if (n_detection > 0):
            
            # Eliminate non-vehicles object
            def eliminate_non_vehicle(series):
                detections = series['object_probs'].split(' ')
                for dtc in detections:
                    if dtc in coco_vehicle_labels:
                        return True
                return False

            objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]
            
        n_detection = objects.shape[0]
        if (n_detection > 0):
            # For every bounding box vehicles, find the bounding box center point    
            centers = objects.apply(lambda r: [
                              (r['left'] + r['right'])/2,
                              (r['top'] + r['bottom'])/2
                              ], axis=1)

            # Transformation the points
            points_np = np.array([i for i in centers])
            points_transform = np.int32(warpPerspectivePoints(points_np, M))
            
            # Map the points to the image
            for i in points_transform:
                cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)

        cv2.imshow('Image', new_map_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
finally:
    cv2.destroyAllWindows()

64